In [20]:
import sys
sys.path.append("/anaconda/envs/azureml_py38/lib/python3.8/site-packages")

In [21]:
import psycopg2
import pandas as pd
import numpy as np

In [22]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [23]:
conn, cur = get_db_cursor()


In [24]:
v1_samples = pd.read_csv('./outputFiles/spectra.csv',usecols=[0])

In [25]:
v1_samples

,Unnamed: 0
0,AGD207-408SA0706
1,AGD207-431SA0001
2,AGD207-431SA0002
3,AGD207-431SA0003
4,AGD207-431SA0004
...,...
87362,CW017SA7164
87363,CW017SA7167
87364,CW017SA7168
87365,CW104SA0052


In [26]:
v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)

/tmp/ipykernel_67100/2923516565.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)


In [27]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
185458,CT021SA2596
185459,CM754SA0025
185460,CD359SA0001
185461,CE178SA0067


In [28]:
v2_samples = v2_samples.loc[~v2_samples['sample_code'].isin(v1_samples['Unnamed: 0'])]

In [29]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
185458,CT021SA2596
185459,CM754SA0025
185460,CD359SA0001
185461,CE178SA0067


In [30]:
sample_codes = v2_samples['sample_code'].values

In [31]:
if(len(sample_codes) < 5000):
    count  = len(sample_codes)
    step=count
elif(len(sample_codes) < 70000):
    count = len(sample_codes)
    step=5000
else:
    count = 70000
    step=5000
start = 0


In [32]:
count

70000

In [33]:
start

0

In [34]:
step

5000

In [35]:
spectra = pd.DataFrame()

In [54]:
def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [37]:
for i in np.arange(start, count, step):
        
    print("Fetching spectra from {}".format(start))
    samples = [i for i in sample_codes][start:start+step]
    print(len(samples))
    query = f"SELECT spectraldata.metadata_id, spectraldata.non_averaged_spectra, spectraldata.averaged_spectra, mandatorymetadata.sample_code FROM spectraldata INNER JOIN mandatorymetadata ON mandatorymetadata.metadata_id = spectraldata.metadata_id WHERE sample_code IN {str(samples).replace('[','(').replace(']',')')}"

    _ = pd.read_sql(query, con=conn)
    print("This is the spectra",_)
    spectra = pd.concat([spectra, _], axis=0)
    start = start + step
    if (count-step) > 5000:
        step=5000
    else:
        step = count-step





Fetching spectra from 0
5000
This is the spectra        metadata_id                               non_averaged_spectra  \
0           179670                                               None   
1           179670  {'521': [2.3906581683078625, 2.38097433499155,...   
2           179670  {'521': [2.1405241051036783, 2.165647650151477...   
3           179670  {'513': [0.0088157016990374, 0.00958802969112,...   
4           179670  {'513': [0.0088157016990374, 0.00958802969112,...   
...            ...                                                ...   
19536       162600  {'522': [2.227432254754312, 2.223466791121651,...   
19537       162602                                               None   
19538       162602  {'522': [2.226552353462885, 2.21668459943883, ...   
19539       162602  {'522': [2.380440041123865, 2.3974703465638414...   
19540       162603                                               None   

                                        averaged_spectra  sample_code  
0 

/tmp/ipykernel_67100/2443097163.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_67100/2443097163.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_67100/2443097163.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_67100/2443097163.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAP

In [38]:
spectra

,metadata_id,non_averaged_spectra,averaged_spectra,sample_code
0,179670,None,"{'521': [2.2655911367057704, 2.273310992571514...",ML-SS21-1
1,179670,"{'521': [2.3906581683078625, 2.38097433499155,...",None,ML-SS21-1
2,179670,"{'521': [2.1405241051036783, 2.165647650151477...",None,ML-SS21-1
3,179670,"{'513': [0.0088157016990374, 0.00958802969112,...",None,ML-SS21-1
4,179670,"{'513': [0.0088157016990374, 0.00958802969112,...",None,ML-SS21-1
...,...,...,...,...
18784,174325,"{'522': [2.314154897949655, 2.283712153409843,...",None,CF177SA4847
18785,174325,None,"{'522': [2.272728247310493, 2.2546004248827813...",CF177SA4847
18786,174326,None,"{'522': [2.1428143427840016, 2.105124066904913...",CF177SA4752
18787,174326,"{'522': [2.1735385023639964, 2.113213803092161...",None,CF177SA4752


In [39]:
df_ = spectra.copy(deep=True)

In [40]:
df_['spectra'] = df_['averaged_spectra'] + df_['non_averaged_spectra']

In [47]:
averaged = df_.loc[df_['non_averaged_spectra'].isna()]
not_averaged = df_.loc[df_['averaged_spectra'].isna()]
averaged['spectra'] = averaged['averaged_spectra']
not_averaged['spectra'] = not_averaged['non_averaged_spectra']
df = pd.concat([averaged, not_averaged])


/tmp/ipykernel_67100/1084805249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  averaged['spectra'] = averaged['averaged_spectra']
/tmp/ipykernel_67100/1084805249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_averaged['spectra'] = not_averaged['non_averaged_spectra']


In [48]:
df

,metadata_id,non_averaged_spectra,averaged_spectra,sample_code,spectra
0,179670,None,"{'521': [2.2655911367057704, 2.273310992571514...",ML-SS21-1,"{'521': [2.2655911367057704, 2.273310992571514..."
5,179712,None,"{'521': [2.2946008508409697, 2.27379168724669,...",ML-SS21-1,"{'521': [2.2946008508409697, 2.27379168724669,..."
6,179754,None,"{'521': [2.235812133945057, 2.3084154307617473...",ML-SS21-1,"{'521': [2.235812133945057, 2.3084154307617473..."
7,179814,None,"{'521': [2.3830898348113863, 2.376923259819746...",ML-SS21-1,"{'521': [2.3830898348113863, 2.376923259819746..."
8,179874,None,"{'522': [2.2885870694773565, 2.306158053862360...",ML-SS21-1,"{'522': [2.2885870694773565, 2.306158053862360..."
...,...,...,...,...,...
18782,174324,"{'522': [2.0846219245290767, 2.1293823581819, ...",None,CF177SA4851,"{'522': [2.0846219245290767, 2.1293823581819, ..."
18783,174325,"{'522': [2.2313015966713303, 2.22548869635572,...",None,CF177SA4847,"{'522': [2.2313015966713303, 2.22548869635572,..."
18784,174325,"{'522': [2.314154897949655, 2.283712153409843,...",None,CF177SA4847,"{'522': [2.314154897949655, 2.283712153409843,..."
18787,174326,"{'522': [2.1735385023639964, 2.113213803092161...",None,CF177SA4752,"{'522': [2.1735385023639964, 2.113213803092161..."


In [51]:
conn.close()
spectra = df[['sample_code', 'spectra']]


In [52]:
spectra = spectra.set_index('sample_code')


In [55]:
spectra = convertSpectra(spectra)


spectra.to_csv("./outputFiles/spectraldata.csv")

Converting spectra


ValueError: 1728 columns passed, passed data had 1739 columns

In [ ]:
import pandas as pd
spectra= pd.read_csv("outputFiles/spectraldata.csv",usecols=[0])
len(spectra)

In [ ]:
spectra.to_csv("outputFiles/samples.csv")